In [4]:
import torch
import torch.nn as nn

class SegNet(nn.Module):
  def __init__(self,num_classes=21,init_weights=True): # init_weights 는 pretrained 모델을 가져오기 위함
    super(SegNet,self).__init__()
    
    def CBR(in_channels,out_channels,kernel_size=3,stride=1,padding=1):
      layers = []
      layers += [nn.Conv2d(in_channels=in_channels,
                          out_channels=out_channels,
                          kernel_size=kernel_size,
                          stride=stride,
                          padding=padding)]
      layers += [nn.BatchNorm2d(num_features=out_channels)]
      layers += [nn.ReLU()]

      cbr = nn.Sequential(*layers) # layers 를 리스트로 만들어서 넣어준 후에 nn.Sequential(*리스트) 를 통해 만들수도 있음(파이썬 문법)
      return cbr

    # conv1 
    self.cbr1_1 = CBR(3, 64, 3, 1, 1)
    self.cbr1_2 = CBR(64, 64, 3, 1, 1)
    self.pool1 = nn.MaxPool2d(2, stride=2, ceil_mode=True, return_indices=True) 

    # conv2 
    self.cbr2_1 = CBR(64, 128, 3, 1, 1)
    self.cbr2_2 = CBR(128, 128, 3, 1, 1)
    self.pool2 = nn.MaxPool2d(2, stride=2, ceil_mode=True, return_indices=True) 

    # conv3
    self.cbr3_1 = CBR(128, 256, 3, 1, 1)
    self.cbr3_2 = CBR(256, 256, 3, 1, 1)
    self.cbr3_3 = CBR(256, 256, 3, 1, 1)
    self.pool3 = nn.MaxPool2d(2, stride=2, ceil_mode=True, return_indices=True) 

    # conv4
    self.cbr4_1 = CBR(256, 512, 3, 1, 1)
    self.cbr4_2 = CBR(512, 512, 3, 1, 1)
    self.cbr4_3 = CBR(512, 512, 3, 1, 1)
    self.pool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True, return_indices=True) 

    # conv5
    self.cbr5_1 = CBR(512, 512, 3, 1, 1)
    self.cbr5_2 = CBR(512, 512, 3, 1, 1)
    self.cbr5_3 = CBR(512, 512, 3, 1, 1)
    self.pool5 = nn.MaxPool2d(2, stride=2, ceil_mode=True, return_indices=True) 

    # deconv5
    self.unpool5 = nn.MaxUnpool2d(2, stride=2)
    self.dcbr5_3 = CBR(512, 512, 3, 1, 1)
    self.dcbr5_2 = CBR(512, 512, 3, 1, 1)
    self.dcbr5_1 = CBR(512, 512, 3, 1, 1)

    # deconv4 
    self.unpool4 = nn.MaxUnpool2d(2, stride=2)
    self.dcbr4_3 = CBR(512, 512, 3, 1, 1)
    self.dcbr4_2 = CBR(512, 512, 3, 1, 1)
    self.dcbr4_1 = CBR(512, 256, 3, 1, 1)

    # deconv3
    self.unpool3 = nn.MaxUnpool2d(2, stride=2)
    self.dcbr3_3 = CBR(256, 256, 3, 1, 1)
    self.dcbr3_2 = CBR(256, 256, 3, 1, 1)
    self.dcbr3_1 = CBR(256, 128, 3, 1, 1)

    # deconv2
    self.unpool2 = nn.MaxUnpool2d(2, stride=2)
    self.dcbr2_2 = CBR(128, 128, 3, 1, 1)
    self.dcbr2_1 = CBR(128, 64, 3, 1, 1)

    # deconv1
    self.unpool1 = nn.MaxUnpool2d(2, stride=2)
    self.deconv1_1 = CBR(64, 64, 3, 1, 1)
    
    # Score
    # self.score_fr = nn.Conv2d(64, num_classes, kernel_size = 1) -> DeconvNet
    self.score_fr = nn.Conv2d(64, num_classes, kernel_size = 3, padding=1)

    if init_weights:
      self._initialize_weights()

  def _initialize_weights(self):
      for m in self.modules():
          if isinstance(m, nn.Conv2d):
              torch.nn.init.xavier_uniform_(m.weight) # 초깃값 설정 https://pytorch.org/docs/stable/nn.init.html , kaiming_normal(he) 도 적용가능

              if m.bias is not None: # 편향이 있다면 편향을 0으로 만들어줌
                  torch.nn.init.zeros_(m.bias)

  def forward(self,x):
    h = self.cbr1_1(x)
    h = self.cbr1_2(h)
    dim1 = h.size()
    h, pool1_indices = self.pool1(h)
    
    h = self.cbr2_1(h)
    h = self.cbr2_2(h)
    dim2 = h.size()
    h, pool2_indices = self.pool2(h)
    
    h = self.cbr3_1(h)
    h = self.cbr3_2(h)
    h = self.cbr3_3(h)
    dim3 = h.size()
    h, pool3_indices = self.pool3(h)
    
    h = self.cbr4_1(h)
    h = self.cbr4_2(h)
    h = self.cbr4_3(h)
    dim4 = h.size()
    h, pool4_indices = self.pool4(h)
    
    h = self.cbr5_1(h)
    h = self.cbr5_2(h)
    h = self.cbr5_3(h)
    dim5 = h.size()
    h, pool5_indices = self.pool5(h)
    
    h = self.unpool5(h, pool5_indices, output_size = dim5)
    h = self.dcbr5_3(h)
    h = self.dcbr5_2(h)
    h = self.dcbr5_1(h)
    
    h = self.unpool4(h, pool4_indices, output_size = dim4)
    h = self.dcbr4_3(h)
    h = self.dcbr4_2(h)
    h = self.dcbr4_1(h)
    
    h = self.unpool3(h, pool3_indices, output_size = dim3)
    h = self.dcbr3_3(h)
    h = self.dcbr3_2(h)
    h = self.dcbr3_1(h)
    
    h = self.unpool2(h, pool2_indices, output_size = dim2)
    h = self.dcbr2_2(h)
    h = self.dcbr2_1(h)
    
    h = self.unpool1(h, pool1_indices, output_size = dim1)
    h = self.deconv1_1(h)
    out = self.score_fr(h) 
    
    return out   

In [5]:
# 구현된 model에 임의의 input을 넣어 output이 잘 나오는지 test
device='cuda' if torch.cuda.is_available() else 'cpu'

model = SegNet(num_classes=12)
x = torch.randn([1, 3, 512, 512])
print("input shape : ", x.shape)
out = model(x).to(device)
print("output shape : ", out.size())


input shape :  torch.Size([1, 3, 512, 512])
output shape :  torch.Size([1, 12, 512, 512])
